Generare cereri folosind modelul RoLlama3-8b-Instruct-DPO



1.   Incarcam Modelul.
2.   Construim un prompt care sa dea indicatii clare despre cum trebuie sa fie formata cererea.
3.   Generam aleator niste "configuratii" de cereri
4.   Avem in vedere partile implicate, scopul si tipul de cerere





In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "OpenLLM-Ro/RoLlama3-8b-Instruct-DPO"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto"
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/539 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

In [ ]:
SYSTEM_MSG = "Ești un asistent care redactează cereri clare, politicoase și bine structurate în limba română, în stil juridic."

TEMPLATE = """
Redactează o cerere juridică în limba română pe baza următoarelor elemente:
- TipCerere: {tip}
- Destinatar: {destinatar}
- Expeditor: {expeditor}
- Scop: {scop}
- Argumente: {argumente}
- Termen dorit: {termen}
- Ton: {ton}
- Lungime aproximativă: {lungime} cuvinte

Asigură un stil juridic clar, coerent și bine structurat, fără a oferi consultanță juridică.
"""

import random

types = [
    "cerere de chemare în judecată",
    "plângere prealabilă",
    "cerere de ordonanță președințială",
    "plângere contravențională",
    "cerere de executare silită",
    "întâmpinare",
    "contestare proces-verbal",
    "petiție către instanță"
]

destinatari = [
    "Judecătoria competentă",
    "Tribunalul competent",
    "Curtea de Apel",
    "Parchetul de pe lângă Judecătorie",
    "executorul judecătoresc",
    "avocatul pârâtului",
    "poliția locală"
]

expeditori = [
    "reclamant",
    "pârât",
    "petent",
    "creditor",
    "debitor",
    "reprezentant legal",
    "întimat"
]

scopuri = [
    "obligarea pârâtului la plata prejudiciului",
    "anularea unui act administrativ",
    "contestarea unei sancțiuni contravenționale",
    "solicitarea suspendării executării",
    "recuperarea unei datorii",
    "constatarea nulității unui contract",
    "oprirea executării silite"
]

argumente = [
    "prezint probele anexate",
    "invoc prevederile legale aplicabile",
    "anexez înscrisuri justificative",
    "menționez circumstanțele faptice relevante",
    "argumentez temeinicia și legalitatea cererii"
]

termeni = [
    "de urgență",
    "în termen legal",
    "până la primul termen de judecată",
    "în cel mult 15 zile",
    "în regim de maximă urgență"
]

tonuri = ["formal", "strict formal"]
lungimi = [300, 400, 500]

def sample_prompt():
    return TEMPLATE.format(
        tip=random.choice(types),
        destinatar=random.choice(destinatari),
        expeditor=random.choice(expeditori),
        scop=random.choice(scopuri),
        argumente=random.choice(argumente),
        termen=random.choice(termeni),
        ton=random.choice(tonuri),
        lungime=random.choice(lungimi)
    )

def call_model(system, prompt, temperature=0.4, max_new_tokens=500):
    full_prompt = f"System: {system}\nUser: {prompt}\nAssistant:"
    inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# GENERARE 3 CERERI JURIDICE ALEATORII
for i in range(18):
    p = sample_prompt()
    print(f"\n=== Prompt {i+1} ===\n{p}")
    out = call_model(SYSTEM_MSG, p)
    print(f"\n--- Cerere generată ---\n{out}\n")



=== Prompt 1 ===

Redactează o cerere juridică în limba română pe baza următoarelor elemente:
- TipCerere: contestare proces-verbal
- Destinatar: poliția locală
- Expeditor: reclamant
- Scop: recuperarea unei datorii
- Argumente: invoc prevederile legale aplicabile
- Termen dorit: în termen legal
- Ton: formal
- Lungime aproximativă: 300 cuvinte

Asigură un stil juridic clar, coerent și bine structurat, fără a oferi consultanță juridică.


--- Cerere generată ---
System: Ești un asistent care redactează cereri clare, politicoase și bine structurate în limba română, în stil juridic.
User: 
Redactează o cerere juridică în limba română pe baza următoarelor elemente:
- TipCerere: contestare proces-verbal
- Destinatar: poliția locală
- Expeditor: reclamant
- Scop: recuperarea unei datorii
- Argumente: invoc prevederile legale aplicabile
- Termen dorit: în termen legal
- Ton: formal
- Lungime aproximativă: 300 cuvinte

Asigură un stil juridic clar, coerent și bine structurat, fără a oferi c

In [ ]:
SYSTEM_MSG = "Ești un asistent care redactează cereri juridice clare, politicoase și bine structurate în limba română, respectând toate cerințele formale impuse de lege."

TEMPLATE = """
Redactează o cerere juridică completă în limba română, pe baza următoarelor elemente:
- TipCerere: {tip} (ex: chemare în judecată, apel, recurs)
- Parti implicate: {parti} (persoane fizice sau juridice)
- Context/Scop: {scop} (ex: reglementare, dovadă, soluționare conflict, constatare fapt)
- Conflict juridic: {conflict} (doar pentru cereri de chemare în judecată)
- Argumente și probe: {argumente}
- Termen dorit: {termen}
- Ton: {ton}
- Lungime aproximativă: {lungime} cuvinte

Structura cererii trebuie să includă:
- Titlu
- Preambul
- Corpul documentului (cu articole/paragrafe numerotate)
- Semnături
- Dată
- Ștampile (dacă este cazul)

Asigură respectarea cerințelor legale și a formalităților prevăzute de lege pentru cererile juridice.
"""

import random

types = ["chemare în judecată", "apel", "recurs"]
parti = ["persoană fizică", "persoană juridică"]
scopuri = [
    "reglementare",
    "dovadă",
    "soluționare a unui conflict",
    "constatare a unui fapt"
]
conflict = ["există conflict juridic", "nu este cazul unui conflict juridic"]  # doar pentru chemare în judecată
argumente = [
    "prezint probele anexate",
    "invoc prevederile legale aplicabile",
    "anexez înscrisuri justificative",
    "menționez circumstanțele faptice relevante"
]
termeni = ["de urgență", "în termen legal", "până la primul termen de judecată"]
tonuri = ["formal", "strict formal"]
lungimi = [150, 200, 100]

def sample_prompt():
    return TEMPLATE.format(
        tip=random.choice(types),
        parti=random.choice(parti),
        scop=random.choice(scopuri),
        conflict=random.choice(conflict),
        argumente=random.choice(argumente),
        termen=random.choice(termeni),
        ton=random.choice(tonuri),
        lungime=random.choice(lungimi)
    )

def call_model(system, prompt, temperature=0.4, max_new_tokens=500):
    full_prompt = f"System: {system}\nUser: {prompt}\nAssistant:"
    inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)




In [ ]:
# GENERARE 18 CERERI JURIDICE ALEATORII
for i in range(18):
    p = sample_prompt()
    print(f"\n=== Prompt {i+1} ===\n{p}")
    out = call_model(SYSTEM_MSG, p)
    print(f"\n--- Cerere generată ---\n{out}\n")


=== Prompt 1 ===

Redactează o cerere juridică completă în limba română, pe baza următoarelor elemente:
- TipCerere: chemare în judecată (ex: chemare în judecată, apel, recurs)
- Parti implicate: persoană fizică (persoane fizice sau juridice)
- Context/Scop: soluționare a unui conflict (ex: reglementare, dovadă, soluționare conflict, constatare fapt)
- Conflict juridic: nu este cazul unui conflict juridic (doar pentru cereri de chemare în judecată)
- Argumente și probe: anexez înscrisuri justificative
- Termen dorit: în termen legal
- Ton: formal
- Lungime aproximativă: 200 cuvinte

Structura cererii trebuie să includă:
- Titlu
- Preambul
- Corpul documentului (cu articole/paragrafe numerotate)
- Semnături
- Dată
- Ștampile (dacă este cazul)

Asigură respectarea cerințelor legale și a formalităților prevăzute de lege pentru cererile juridice.


--- Cerere generată ---
System: Ești un asistent care redactează cereri juridice clare, politicoase și bine structurate în limba română, respe

In [ ]:
# GENERARE 3 CERERI JURIDICE ALEATORII
for i in range(3):
    p = sample_prompt()
    print(f"\n=== Prompt {i+1} ===\n{p}")
    out = call_model(SYSTEM_MSG, p)
    print(f"\n--- Cerere generată ---\n{out}\n")


=== Prompt 1 ===

Redactează o cerere juridică completă în limba română, pe baza următoarelor elemente:
- TipCerere: recurs (ex: chemare în judecată, apel, recurs)
- Parti implicate: persoană fizică (persoane fizice sau juridice)
- Context/Scop: reglementare (ex: reglementare, dovadă, soluționare conflict, constatare fapt)
- Conflict juridic: nu este cazul unui conflict juridic (doar pentru cereri de chemare în judecată)
- Argumente și probe: prezint probele anexate
- Termen dorit: de urgență
- Ton: formal
- Lungime aproximativă: 150 cuvinte

Structura cererii trebuie să includă:
- Titlu
- Preambul
- Corpul documentului (cu articole/paragrafe numerotate)
- Semnături
- Dată
- Ștampile (dacă este cazul)

Asigură respectarea cerințelor legale și a formalităților prevăzute de lege pentru cererile juridice.


--- Cerere generată ---
System: Ești un asistent care redactează cereri juridice clare, politicoase și bine structurate în limba română, respectând toate cerințele formale impuse de l

Generam mai multe cereri pentru a putea sa le trimit mai departe catre llm-ul evaluator


Am creat un JSON cu date de forma:{prompt: , text_generat: } pentru a usura procesul de evaluare
